# Import libraries and packages

In [ ]:
!pip install category_encoders
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 24.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import plotly.express as px


In [ ]:
# Mount the drive
drive.mount('/content/drive')

# Path to the directory
project_dir = '/content/drive/MyDrive/IHU - Data Science/Thesis/Dataset/'

# Change the current working directory to this folder
os.chdir(project_dir)

# Verify the current working directory
print("Current Working Directory:", os.getcwd())

# Read the files
train_data = pd.read_csv('train.csv')
fulfilment_center_data = pd.read_csv('fulfilment_center_info.csv')
meal_data = pd.read_csv('meal_info.csv')

merged_df = pd.merge(train_data, fulfilment_center_data, on='center_id')
final_merged_df = pd.merge(merged_df, meal_data, on='meal_id')

Mounted at /content/drive
Current Working Directory: /content/drive/MyDrive/IHU - Data Science/Thesis/Dataset


# Demand Forecasting Process

In [ ]:
df_additional_features = final_merged_df.copy()

# Food Category and Center Type Combination
df_additional_features['NewCategoryCenter'] = df_additional_features['category'] + '_' + df_additional_features['center_type'].astype(str)
# Cuisine and Center combination
df_additional_features['NewCuisineCenter'] = df_additional_features['cuisine'] + '_' + df_additional_features['center_type'].astype(str)

# Slice the data in chronological order to avoid data leakage
train_df = df_additional_features[df_additional_features["week"] < 136]
test_df = df_additional_features[df_additional_features["week"] >= 136]

train_df = train_df.copy()
test_df = test_df.copy()
train_df.drop(columns=['id'], inplace=True)
test_df.drop(columns=['id'], inplace=True)


In [ ]:
# Identify categorical features
categorical_features = ['center_id', 'meal_id', 'city_code', 'region_code', 'center_type',
                        'category', 'cuisine', 'op_area', 'NewCategoryCenter', 'NewCuisineCenter']

# Initialize OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Use sparse_output=False

# Fit the encoder on the categorical features from the training set
one_hot_encoder.fit(train_df[categorical_features])

# Transform the training and test sets
train_encoded = one_hot_encoder.transform(train_df[categorical_features])
test_encoded = one_hot_encoder.transform(test_df[categorical_features])

# Convert the encoded data to DataFrame, preserving the index
train_encoded_df = pd.DataFrame(train_encoded,
                                columns=one_hot_encoder.get_feature_names_out(categorical_features),
                                index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded,
                               columns=one_hot_encoder.get_feature_names_out(categorical_features),
                               index=test_df.index)

# Drop original categorical columns and concatenate the one-hot encoded columns
train_df_encoded = pd.concat([train_df.drop(columns=categorical_features), train_encoded_df], axis=1)
test_df_encoded = pd.concat([test_df.drop(columns=categorical_features), test_encoded_df], axis=1)

# Check for consistent rows
print("Train shape:", train_df_encoded.shape)
print("Test shape:", test_df_encoded.shape)


Train shape: (423727, 298)
Test shape: (32821, 298)


In [ ]:
# Split the data accordingly into train and test set
X_train, y_train = train_df_encoded.drop(columns=['num_orders']), train_df_encoded['num_orders']
X_test, y_test = test_df_encoded.drop(columns=['num_orders']), test_df_encoded['num_orders']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((423727, 297), (423727,), (32821, 297), (32821,))

In [ ]:
# Convert the training and test data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Initialize and train the XGBoost regressor
xgb_params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'eval_metric': 'rmse',
    'device': 'cuda',
    'learning_rate': 0.1,
    'max_depth': 7,
    'random_state': 42
}
%time xgb_regressor = xgb.train(xgb_params, dtrain, num_boost_round=200)

# Predict order quantities on the test set
y_pred = xgb_regressor.predict(dtest)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Root Mean Squared Error"
mae = mean_absolute_error(y_test, y_pred)
mape = (abs((y_test - y_pred) / y_test).mean()) * 100  # Mean Absolute Percentage Error
r2 = r2_score(y_test, y_pred)

# Create a dictionary with the metrics
metrics_data = {
    "Metric": [
        "Mean Absolute Error (MAE)",
        "Mean Absolute Percentage Error (MAPE)",
        "Root Mean Squared Error (RMSE)",
        "R² Score"
    ],
    "Value": [
        f"{mae:.2f}",
        f"{mape:.2f}%",
        f"{rmse:.2f}",
        f"{r2:.2f}"
    ]
}

# Convert to a pandas DataFrame
metrics_df = pd.DataFrame(metrics_data)

# Display the table
print(metrics_df)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:16:21] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:16:21] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


CPU times: user 1min 8s, sys: 251 ms, total: 1min 9s
Wall time: 41.5 s
                                  Metric   Value
0              Mean Absolute Error (MAE)   76.72
1  Mean Absolute Percentage Error (MAPE)  66.02%
2         Root Mean Squared Error (RMSE)  124.92
3                               R² Score    0.80


In [ ]:
# Create a variable to store the test_df
data = test_df.copy()

# Convert the y_pred variable from numpy array to pandas dataframe for easy concatenation
y_pred_series = pd.Series(y_pred, index=data.index, name='predicted')

# Concatenate the test data with actual and predicted values
data = pd.concat([data, y_pred_series], axis=1)

data.drop(['num_orders'], axis= 1, inplace=True)

# LP Process

## Main Process

In [ ]:
data = data[['week', 'center_id', 'meal_id', 'checkout_price', 'predicted']]

# Preprocess the data to extract relevant information for the optimization model
centers = data['center_id'].unique()  # Unique center IDs
print("Available Center IDs:", centers)

# User selects centers
selected_centers = input("Enter the center IDs you want to include (comma-separated): ")
try:
    selected_centers = [int(center.strip()) for center in selected_centers.split(',') if int(center.strip()) in centers]
except ValueError:
    print("Invalid input! Please enter valid center IDs.")
    exit()

# Filter data for selected centers
filtered_data_center = data[data['center_id'].isin(selected_centers)]


# Group by 'center_id' and get unique meal IDs for each center
center_meals = filtered_data_center.groupby('center_id')['meal_id'].unique()

# Find the intersection of meal IDs across all centers
common_meals = set(center_meals.iloc[0])  # Start with the meals from the first center
for meals in center_meals[1:]:  # Iterate over the remaining centers
    common_meals &= set(meals)  # Keep only meals that are common to all centers

# Convert the common meals back to a list (or array) if needed
common_meals = list(common_meals)

print("\nAvailable Common Meal IDs:", common_meals)

# User selects meals
selected_meals = input(f"Enter the meal IDs you want to include (comma-separated): ")
try:
    selected_meals = [int(meal.strip()) for meal in selected_meals.split(',') if int(meal.strip()) in common_meals]
except ValueError:
    print("Invalid input! Please enter valid meal IDs.")
    exit()

# Function to check meal availability over 10 weeks within the specified week range (136 to 145)
def is_meal_available_for_10_weeks(meal_id, center_id, data):
    # Filter data for the specific center and meal_id and within the weeks 136 to 145
    filtered_data = data[(data['meal_id'] == meal_id) &
                          (data['center_id'] == center_id) &
                          (data['week'] >= 136) &
                          (data['week'] <= 145)]
    # Get the number of distinct weeks the meal is available for within the specified range
    available_weeks = filtered_data['week'].nunique()
    return available_weeks >= 10


valid_meals = []
invalid_meals_details = []  # To store invalid meals with centers where they are invalid

for meal in selected_meals:
    invalid_centers = [center for center in selected_centers if not is_meal_available_for_10_weeks(meal, center, filtered_data_center)]
    if not invalid_centers:
        valid_meals.append(meal)
    else:
        invalid_meals_details.append({'meal': meal, 'invalid_centers': invalid_centers})

if invalid_meals_details:
    print("Invalid meals and their invalid centers:")
    for detail in invalid_meals_details:
        print(f"Meal: {detail['meal']} is not valid in centers: {detail['invalid_centers']}")
else:
    print("All selected meals are valid in all centers.")

answer = input(f"Want to proceed with the valid meals {valid_meals}? (type Y or N):")
if answer == "N":
    print("Exiting as per user request...")
    sys.exit()

selected_meals = valid_meals

# Filter data for selected meals
filtered_data_meal = filtered_data_center[filtered_data_center['meal_id'].isin(selected_meals)]


# Collect ordering, holding, and unit costs for each meal
ordering_cost = {}
holding_cost = {}
unit_cost = {}

for meal_id in selected_meals:
    print(f"\nFor Meal ID {meal_id}:")
    ordering_cost[meal_id] = float(input(f"  Enter ordering cost for Meal {meal_id}: "))
    holding_cost[meal_id] = float(input(f"  Enter holding cost per unit for Meal {meal_id}: "))
    unit_cost[meal_id] = float(input(f"  Enter unit cost for Meal {meal_id}: "))


# Create dictionaries for demand and checkout price, indexed by meal_id and week
demand = {}
checkout_price = {}

for center_id in selected_centers:
    center_data = filtered_data_meal[filtered_data_meal['center_id'] == center_id]  # Filter data for the center

    demand_temp = {}
    checkout_price_temp = {}

    for meal_id in selected_meals:
        meal_data = center_data[center_data['meal_id'] == meal_id]  # Filter data for the meal

        # Store predicted demand and checkout prices for each week
        demand_temp[meal_id] = meal_data['predicted'].tolist()
        checkout_price_temp[meal_id] = meal_data['checkout_price'].tolist()

    demand[center_id] = demand_temp
    checkout_price[center_id] = checkout_price_temp

weeks = list(range(1, len(filtered_data_meal['week'].unique().tolist())+1))  # List of weeks
meals = selected_meals
centers = selected_centers

# Define baseline budget using realistic weekly demand
baseline_budget = {}
margin = 1.2  # Additional margin to account for extra costs

for center in centers:
    total_demand_cost = 0
    for meal in meals:
        weekly_demand = demand[center][meal]  # Predicted demand per week
        for t, demand_value in enumerate(weekly_demand):
            total_demand_cost += unit_cost[meal] * demand_value
    baseline_budget[center] = total_demand_cost / (len(weeks) - 1) * margin  # Adjusted with margin

# Output the calculated baseline budgets
print("\nSufficient Budget Suggestion by Center with 1.2 margin:")
for center, budget_value in baseline_budget.items():
    print(f"  Center {center}: {budget_value:.2f}")

# Collect user-defined budget and warehouse capacity for each center
budget = {}
warehouse_capacity = {}
initial_inventory = {}

for center_id in selected_centers:
    budget[center_id] = float(input(f"\nEnter the weekly budget for center {center_id}: "))
    warehouse_capacity[center_id] = int(input(f"Enter the warehouse capacity for {center_id}: "))

    initial_inventory_temp = {}
    for meal_id in selected_meals:
        initial_inventory_temp[meal_id] = int(input(f"  Enter initial inventory for Meal {meal_id} in center {center_id}: "))

    initial_inventory[center_id] = initial_inventory_temp

lead_time = 1 # Define lead time as constant

# Initialize the Linear Programming (LP) problem
prob = LpProblem("Multiple_Centers_Meals_Optimization", LpMinimize)

# Decision variables
order_qty = LpVariable.dicts("Order", (centers, meals, weeks), lowBound=0, cat="Integer")
inventory = LpVariable.dicts("Inventory", (centers, meals, weeks), lowBound=0, cat="Integer")
order_indicator = LpVariable.dicts("OrderIndicator", (centers, meals, weeks), cat="Binary")
unmet_demand = LpVariable.dicts("UnmetDemand", (centers, meals, weeks), lowBound=0, cat="Integer")

# Objective function: Minimize ordering, holding, and penalty costs
prob += lpSum(
    ordering_cost[meal] * order_indicator[center][meal][t] +
    order_qty[center][meal][t] * unit_cost[meal] +
    holding_cost[meal] * inventory[center][meal][t] +
    checkout_price[center][meal][t-1] * unmet_demand[center][meal][t]
    for center in centers for meal in meals for t in weeks
)

# Add constraints for inventory balance, order capacity, and budget limits
for center in centers:
    for t in weeks:
        for meal in meals:
            # Inventory balance constraint
            if t == 1:
                prob += inventory[center][meal][t] == initial_inventory[center][meal] - demand[center][meal][t-1] + unmet_demand[center][meal][t]
            else:
                prob += inventory[center][meal][t] == inventory[center][meal][t-1] + order_qty[center][meal][t-lead_time] - demand[center][meal][t-1] + unmet_demand[center][meal][t] - unmet_demand[center][meal][t-1]

            # Link binary variable to order quantity using Big-M method
            prob += order_qty[center][meal][t] <= 1000000 * order_indicator[center][meal][t]

        # Weekly budget constraint for all meals in the center
        prob += lpSum(order_qty[center][meal][t] * unit_cost[meal] + ordering_cost[meal] * order_indicator[center][meal][t] for meal in meals) <= budget[center]

        # Warehouse capacity constraint
        prob += lpSum(inventory[center][meal][t] for meal in meals) <= warehouse_capacity[center]


# Solve the optimization problem
prob.solve()

# Collect results
results = []

for center in centers:
    for meal in meals:
        for t in weeks:
            results.append({
                "Center": center,
                "Week": t,
                "Meal": meal,
                "Order": order_qty[center][meal][t].varValue,
                "Inventory": inventory[center][meal][t].varValue,
                "Unmet Demand": unmet_demand[center][meal][t].varValue
            })

# Check solution status and display results
solution_status = LpStatus[prob.status]
results_df = pd.DataFrame(results)

print(f"Solution Status: {solution_status}")
print("Optimization Results:")
print(results_df)


#  metrics
total_ordering_cost = 0
total_holding_cost = 0
total_penalty_cost = 0
total_unmet_demand = 0
unmet_demand_frequency = 0
budget_utilization = {}

for center in centers:
    center_spending = 0  # Total spending for the center
    for meal in meals:

        unmet_demand_value = unmet_demand[center][meal][len(weeks)].varValue # Total unmet demand for the meal
        total_unmet_demand += unmet_demand_value

        for t in weeks:
            order_cost = ordering_cost[meal] * order_indicator[center][meal][t].varValue + order_qty[center][meal][t].varValue * unit_cost[meal]
            holding_cost_week = holding_cost[meal] * inventory[center][meal][t].varValue
            penalty_cost_week = checkout_price[center][meal][t-1] * unmet_demand[center][meal][t].varValue

            # Accumulate costs
            total_ordering_cost += order_cost
            total_holding_cost += holding_cost_week
            total_penalty_cost += penalty_cost_week

            # Count unmet demand frequency
            if unmet_demand[center][meal][t].varValue > 0:
                unmet_demand_frequency += 1

            center_spending += order_cost

    # Calculate budget utilization for the center
    budget_utilization[center] = round(center_spending / (budget[center] * (len(weeks) - 1)), 2)

# Display metrics
print("\n=== Evaluation Metrics ===")
print(f"Total Cost: {total_ordering_cost + total_holding_cost + total_penalty_cost:.2f}")
print(f"  - Ordering Cost: {total_ordering_cost:.2f}")
print(f"  - Holding Cost: {total_holding_cost:.2f}")
print(f"  - Penalty Cost: {total_penalty_cost:.2f}")
print(f"Total Volume of Unmet Demand By The End Period: {total_unmet_demand:.2f}")
print(f"Unmet Demand Frequency: {unmet_demand_frequency} weeks")
print("\nBudget Utilization by Center:")
for center, utilization in budget_utilization.items():
    print(f"  - Center {center}: {utilization * 100:.2f}%")


Available Center IDs: [ 55  24  11  83  32  13 109  52  93 186 146  57 149  89 124 152  97  74
 108  99  66  94  91  20  34 137  92 126  36 162  75 177  27 157 106  64
 129  14  17 153 139 161  81  26  73  50 104  42 113 145  53  72  67 174
  29  77  41  30  76  59  88 143  58  10 101  80  43  65  39 102 110 132
  23  86  68  51  61]
Enter the center IDs you want to include (comma-separated): 55,24

Available Common Meal IDs: [2304, 2306, 1543, 2569, 2826, 1803, 2956, 2444, 2704, 2577, 2322, 2707, 2581, 1558, 1311, 1571, 1445, 1062, 1962, 1198, 1971, 2867, 1207, 2490, 1727, 1216, 2760, 1993, 1230, 2126, 2640, 1109, 1878, 1754, 2139, 1885, 2664, 1770, 2539, 1902, 1778, 2290, 1525]
Enter the meal IDs you want to include (comma-separated): 1885,1993
All selected meals are valid in all centers.
Want to proceed with the valid meals [1885, 1993]? (type Y or N):Y

For Meal ID 1885:
  Enter ordering cost for Meal 1885: 100
  Enter holding cost per unit for Meal 1885: 10
  Enter unit cost for M

### Visualization

In [ ]:
# Get unique centers and meals
unique_centers = results_df["Center"].unique()
unique_meals = results_df["Meal"].unique()

# Use a qualitative color palette for more distinct colors
color_palette = px.colors.qualitative.Plotly
num_colors = len(color_palette)
color_mapping = {meal: color_palette[i % num_colors] for i, meal in enumerate(unique_meals)}


# Create subplots: 1 column per center
num_rows = 1
num_cols = len(unique_centers)
fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    subplot_titles=[f"Center {center}" for center in unique_centers],
    shared_xaxes=True,
)

# Plot data for each center
for i, center in enumerate(unique_centers):
    center_data = results_df[results_df["Center"] == center]
    for meal in unique_meals:
        meal_data = center_data[center_data["Meal"] == meal]

        # Ensure legendgroup is a string
        legend_group = str(meal)

        # Add bars for unmet demand
        fig.add_trace(
            go.Bar(
                x=meal_data["Week"],
                y=meal_data["Unmet Demand"],  # Assuming 'Unmet Demand' column exists
                name=f"Unmet Demand for {meal}",
                marker=dict(color=color_mapping[meal], opacity=0.6),  # Transparent bars
                legendgroup=legend_group,  # Link traces by meal (as string)
                showlegend=(i == 0),  # Show legend only once per meal
            ),
            row=1,
            col=i + 1,
        )

        # Add lines for order quantities
        fig.add_trace(
            go.Scatter(
                x=meal_data["Week"],
                y=meal_data["Order"],
                mode="lines+markers",
                name=f"Order Quantity for {meal}",
                line=dict(width=2, color=color_mapping[meal]),
                legendgroup=legend_group,  # Link traces by meal (as string)
                showlegend=(i == 0),  # Show legend only once per meal
            ),
            row=1,
            col=i + 1,
        )

# Update layout
fig.update_layout(
    title="Order Quantities and Unmet Demand Over Time by Meal and Center",
    xaxis_title="Weeks",
    yaxis_title="Order Quantity And Unmet Demand",
    height=500,  # Adjust height based on the number of centers
    legend_title="Legend",
    template="plotly"
)

# Show the figure
fig.show()


## Cost Reduction Sensitivity Analysis

In [ ]:
budget_reduction_levels = input("Enter budget reduction percentage levels (comma-seperated): ")
budget_reduction_levels = [float(budget.strip()) for budget in budget_reduction_levels.split(',')]

# Store results for sensitivity analysis
sensitivity_results = []

# Loop over different budget reductions
for reduction in budget_reduction_levels:

    # Adjust budget based on reduction percentage
    adjusted_budget = {center: budget[center] * (1 - reduction / 100) for center in centers}

    # Initialize LP problem
    prob = LpProblem("Multiple_Centers_Meals_Optimization", LpMinimize)

    # Decision variables
    order_qty = LpVariable.dicts("Order", (centers, meals, weeks), lowBound=0, cat="Integer")
    inventory = LpVariable.dicts("Inventory", (centers, meals, weeks), lowBound=0, cat="Integer")
    order_indicator = LpVariable.dicts("OrderIndicator", (centers, meals, weeks), cat="Binary")
    unmet_demand = LpVariable.dicts("UnmetDemand", (centers, meals, weeks), lowBound=0, cat="Integer")

    # Objective function: Minimize ordering + holding + penalty costs
    prob += lpSum(
        ordering_cost[meal] * order_indicator[center][meal][t] +
        order_qty[center][meal][t] * unit_cost[meal] +
        holding_cost[meal] * inventory[center][meal][t] +
        checkout_price[center][meal][t-1] * unmet_demand[center][meal][t]
        for center in centers for meal in meals for t in weeks
    )

    # Constraints
    for center in centers:
        for t in weeks:
            for meal in meals:
                # Inventory balance constraint
                if t == 1:
                    prob += inventory[center][meal][t] == initial_inventory[center][meal] - demand[center][meal][t-1] + unmet_demand[center][meal][t]
                else:
                    if t - lead_time > 0:
                        prob += inventory[center][meal][t] == inventory[center][meal][t-1] + order_qty[center][meal][t-lead_time] - demand[center][meal][t-1] + unmet_demand[center][meal][t] - unmet_demand[center][meal][t-1]
                    else:
                        prob += inventory[center][meal][t] == inventory[center][meal][t-1] - demand[center][meal][t-1] + unmet_demand[center][meal][t] - unmet_demand[center][meal][t-1]

                # Link binary variable to order quantity
                prob += order_qty[center][meal][t] <= 1000000 * order_indicator[center][meal][t]

            # Weekly budget constraint with adjusted budget
            prob += lpSum(order_qty[center][meal][t] * unit_cost[meal] + ordering_cost[meal] * order_indicator[center][meal][t] for meal in meals) <= adjusted_budget[center]

            # Warehouse capacity constraint
            prob += lpSum(inventory[center][meal][t] for meal in meals) <= warehouse_capacity[center]

    # Solve the optimization problem
    prob.solve()

    # Calculate evaluation metrics
    total_ordering_cost = 0
    total_holding_cost = 0
    total_penalty_cost = 0
    total_unmet_demand = 0
    unmet_demand_frequency = 0
    budget_utilization = {}

    for center in centers:
        center_spending = 0
        for meal in meals:
            unmet_demand_value = unmet_demand[center][meal][len(weeks)].varValue
            total_unmet_demand += unmet_demand_value

            for t in weeks:
                order_cost = ordering_cost[meal] * order_indicator[center][meal][t].varValue + order_qty[center][meal][t].varValue * unit_cost[meal]
                holding_cost_week = holding_cost[meal] * inventory[center][meal][t].varValue
                penalty_cost_week = checkout_price[center][meal][t-1] * unmet_demand[center][meal][t].varValue

                total_ordering_cost += order_cost
                total_holding_cost += holding_cost_week
                total_penalty_cost += penalty_cost_week

                if unmet_demand[center][meal][t].varValue > 0:
                    unmet_demand_frequency += 1

                center_spending += order_cost

        budget_utilization[center] = round(center_spending / (adjusted_budget[center] * (len(weeks) -1)), 2)

    # Store results for this reduction level
    sensitivity_results.append({
        "Reduction %": reduction,
        "Total Cost": total_ordering_cost + total_holding_cost + total_penalty_cost,
        "Ordering Cost": total_ordering_cost,
        "Holding Cost": total_holding_cost,
        "Penalty Cost": total_penalty_cost,
        "Total Unmet Demand": total_unmet_demand,
        "Unmet Demand Frequency": unmet_demand_frequency,
        "Budget Utilization": budget_utilization
    })

# Convert results to DataFrame and display
sensitivity_df = pd.DataFrame(sensitivity_results)
print("\n=== Sensitivity Analysis Results ===")
sensitivity_df


Enter budget reduction percentage levels (comma-seperated): 0,10,20,30,40,50

=== Sensitivity Analysis Results ===


,Reduction %,Total Cost,Ordering Cost,Holding Cost,Penalty Cost,Total Unmet Demand,Unmet Demand Frequency,Budget Utilization
0,0.0,8.208704e+05,196294.385099,0.00000,6.245760e+05,0.00000,9,"{55: 0.87, 24: 0.85}"
1,10.0,1.074560e+06,196294.383899,382.22343,8.778835e+05,0.00000,14,"{55: 0.96, 24: 0.94}"
2,20.0,2.342426e+06,184312.800385,0.00000,2.158113e+06,1198.14645,22,"{55: 1.0, 24: 1.0}"
3,30.0,4.074021e+06,161273.700096,0.00000,3.912748e+06,3502.03344,22,"{55: 1.0, 24: 1.0}"
4,40.0,5.805617e+06,138234.599917,0.00000,5.667382e+06,5805.92040,22,"{55: 1.0, 24: 1.0}"
5,50.0,7.537212e+06,115195.499928,0.00000,7.422016e+06,8109.80740,22,"{55: 1.0, 24: 1.0}"


### Cost Reduction Sensitivity Analysis Plot

In [ ]:
# Extracting data from the results DataFrame
x = sensitivity_df['Reduction %']
ordering = sensitivity_df['Ordering Cost']
holding = sensitivity_df['Holding Cost']
penalty = sensitivity_df['Penalty Cost']
total_unmet = sensitivity_df['Total Unmet Demand']

# Create a Plotly figure
fig = go.Figure()

# Add traces for cost components (primary y-axis)
fig.add_trace(go.Scatter(x=x, y=ordering, mode='lines+markers', name='Ordering Cost', line=dict(color='skyblue')))

# Add a trace for unmet demand (secondary y-axis)
fig.add_trace(go.Scatter(
    x=x, y=total_unmet, mode='lines+markers', name='Unmet Deamnd',
    line=dict(color='red', dash='dash'), yaxis='y2'
))

# Add a trace for unmet demand (secondary y-axis)
fig.add_trace(go.Scatter(
    x=x, y=holding, mode='lines+markers', name='Holding Cost',
    line=dict(color='green', dash='dash'), yaxis='y2'
))

# Update layout for dual y-axes
fig.update_layout(
    title='Cost Components and Unmet Demand vs Budget Reduction',
    xaxis=dict(title='Budget Reduction (%)'),
    yaxis=dict(title='Ordering Cost', titlefont=dict(color='black'), tickfont=dict(color='black')),
    yaxis2=dict(
        title='Unmet Demand and Holding Cost',
        titlefont=dict(color='black'),
        tickfont=dict(color='black'),
        anchor='x', overlaying='y', side='right'
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h'),  # Place legend below the plot
    template='plotly_white'  # Use a clean white theme
)

# Show the plot
fig.show()


## Demand Variation Simulation Analysis

In [ ]:
simulation_scenario = input("Enter demand scenarios with a decimal point e.g., 0.1, 0.2 (comma-seperated): ")
simulation_scenario = [float(scenario.strip()) for scenario in simulation_scenario.split(',')]

simulation_number = int(input("Enter the number of simulations to run for each scenario: "))

# Simulated datasets
simulated_datasets = []

for scenario in simulation_scenario:
    for i in range(simulation_number):
        simulated_data = filtered_data_meal.copy()
        simulated_data['predicted'] = filtered_data_meal['predicted'] * (
            1 + np.random.uniform(-scenario, scenario, len(filtered_data_meal))
        )
        simulated_data['scenario'] = f"Scenario: {scenario*100:.0f}% | Sim {i+1}"
        simulated_datasets.append(simulated_data)


# Combine all simulations into one DataFrame
simulated_data_full = pd.concat(simulated_datasets, ignore_index=True)


# Placeholder for storing metrics across simulations
simulation_results = []


# Optimization Model for Each Simulation
for scenario, scenario_data in simulated_data_full.groupby('scenario'):
    # Use your existing optimization model with `scenario_data` as input
    demand = {
        center: {
            meal: scenario_data[(scenario_data['center_id'] == center) & (scenario_data['meal_id'] == meal)]['predicted'].tolist()
            for meal in meals
        } for center in centers
    }



    # Initialize LP problem
    prob = LpProblem("Multiple_Centers_Meals_Optimization", LpMinimize)

    # Decision variables
    order_qty = LpVariable.dicts("Order", (centers, meals, weeks), lowBound=0, cat="Integer")
    inventory = LpVariable.dicts("Inventory", (centers, meals, weeks), lowBound=0, cat="Integer")
    order_indicator = LpVariable.dicts("OrderIndicator", (centers, meals, weeks), cat="Binary")
    unmet_demand = LpVariable.dicts("UnmetDemand", (centers, meals, weeks), lowBound=0, cat="Integer")

    # Objective function: Minimize ordering + holding + penalty costs
    prob += lpSum(
        ordering_cost[meal] * order_indicator[center][meal][t] +
        order_qty[center][meal][t] * unit_cost[meal] +
        holding_cost[meal] * inventory[center][meal][t] +
        checkout_price[center][meal][t-1] * unmet_demand[center][meal][t]
        for center in centers for meal in meals for t in weeks
    )

    # Constraints
    for center in centers:
        for t in weeks:
            for meal in meals:
                # Inventory balance constraint
                if t == 1:
                    prob += inventory[center][meal][t] == initial_inventory[center][meal] - demand[center][meal][t-1] + unmet_demand[center][meal][t]
                else:
                    if t - lead_time > 0:
                        prob += inventory[center][meal][t] == inventory[center][meal][t-1] + order_qty[center][meal][t-lead_time] - demand[center][meal][t-1] + unmet_demand[center][meal][t] - unmet_demand[center][meal][t-1]
                    else:
                        prob += inventory[center][meal][t] == inventory[center][meal][t-1] - demand[center][meal][t-1] + unmet_demand[center][meal][t] - unmet_demand[center][meal][t-1]

                # Link binary variable to order quantity
                prob += order_qty[center][meal][t] <= 1000000 * order_indicator[center][meal][t]

            # Weekly budget constraint with adjusted budget
            prob += lpSum(order_qty[center][meal][t] * unit_cost[meal] + ordering_cost[meal] * order_indicator[center][meal][t] for meal in meals) <= budget[center]

            # Warehouse capacity constraint
            prob += lpSum(inventory[center][meal][t] for meal in meals) <= warehouse_capacity[center]

    # Solve the optimization problem
    prob.solve()


    # Calculate evaluation metrics
    total_ordering_cost = 0
    total_holding_cost = 0
    total_penalty_cost = 0
    total_unmet_demand = 0
    unmet_demand_frequency = 0
    budget_utilization = {}

    for center in centers:
        center_spending = 0
        for meal in meals:
            unmet_demand_value = unmet_demand[center][meal][len(weeks)].varValue
            total_unmet_demand += unmet_demand_value

            for t in weeks:
                order_cost = ordering_cost[meal] * order_indicator[center][meal][t].varValue + order_qty[center][meal][t].varValue * unit_cost[meal]
                holding_cost_week = holding_cost[meal] * inventory[center][meal][t].varValue
                penalty_cost_week = checkout_price[center][meal][t-1] * unmet_demand[center][meal][t].varValue

                total_ordering_cost += order_cost
                total_holding_cost += holding_cost_week
                total_penalty_cost += penalty_cost_week

                if unmet_demand[center][meal][t].varValue > 0:
                    unmet_demand_frequency += 1

                center_spending += order_cost

        budget_utilization[center] = round(center_spending / (budget[center] * (len(weeks) -1)), 2)

    # Store results for this reduction level
    simulation_results.append({
        "Scenario": scenario,
        "Total Cost": total_ordering_cost + total_holding_cost + total_penalty_cost,
        "Ordering Cost": total_ordering_cost,
        "Holding Cost": total_holding_cost,
        "Penalty Cost": total_penalty_cost,
        "Total Unmet Demand": total_unmet_demand,
        "Unmet Demand Frequency": unmet_demand_frequency,
        "Budget Utilization": budget_utilization
    })

# Convert results to DataFrame and display
simulation_df = pd.DataFrame(simulation_results)
print("\n=== Simulation Analysis Results ===")
simulation_df


Enter demand scenarios with a decimal point e.g., 0.1, 0.2 (comma-seperated): 0,0.1,0.2,0.3,0.4,0.5
Enter the number of simulations to run for each scenario: 100

=== Simulation Analysis Results ===


,Scenario,Total Cost,Ordering Cost,Holding Cost,Penalty Cost,Total Unmet Demand,Unmet Demand Frequency,Budget Utilization
0,Scenario: 0% | Sim 1,8.208704e+05,196294.385099,0.000000,6.245760e+05,0.0,9,"{55: 0.87, 24: 0.85}"
1,Scenario: 0% | Sim 10,8.208704e+05,196294.385099,0.000000,6.245760e+05,0.0,9,"{55: 0.87, 24: 0.85}"
2,Scenario: 0% | Sim 100,8.208704e+05,196294.385099,0.000000,6.245760e+05,0.0,9,"{55: 0.87, 24: 0.85}"
3,Scenario: 0% | Sim 11,8.208704e+05,196294.385099,0.000000,6.245760e+05,0.0,9,"{55: 0.87, 24: 0.85}"
4,Scenario: 0% | Sim 12,8.208704e+05,196294.385099,0.000000,6.245760e+05,0.0,9,"{55: 0.87, 24: 0.85}"
...,...,...,...,...,...,...,...,...
595,Scenario: 50% | Sim 95,1.198915e+06,184653.119774,0.000000,1.014262e+06,0.0,9,"{55: 0.77, 24: 0.81}"
596,Scenario: 50% | Sim 96,8.088716e+05,190954.071472,1051.798500,6.168657e+05,0.0,10,"{55: 0.91, 24: 0.81}"
597,Scenario: 50% | Sim 97,1.631996e+06,208322.731044,7243.880985,1.416429e+06,0.0,9,"{55: 0.89, 24: 0.91}"
598,Scenario: 50% | Sim 98,9.876131e+05,195577.829122,4930.975970,7.871043e+05,0.0,11,"{55: 0.9, 24: 0.84}"


### Simulation Plot



In [ ]:
# Extract relevant columns
simulation_plot_data = simulation_df[['Scenario', 'Unmet Demand Frequency']]

# Convert to DataFrame
df_simulation_plot = pd.DataFrame(simulation_plot_data)

# Extract variation type from the scenario string
df_simulation_plot['Scenario'] = df_simulation_plot['Scenario'].apply(lambda x: x.split(' | ')[0])

# Group by Variation and calculate the mean unmet demand frequency
grouped_data = df_simulation_plot.groupby('Scenario')['Unmet Demand Frequency'].mean().reset_index()

# Create an interactive line plot using Plotly
fig = px.line(
    grouped_data,
    x='Scenario',
    y='Unmet Demand Frequency',
    title='Unmet Demand Frequency by Scenario',
    labels={'Scenario': 'Scenario', 'Unmet Demand Frequency': 'Average Unmet Demand Frequency'},
    markers=True
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Scenario', tickangle=45),
    yaxis=dict(title='Average Unmet Demand Frequency'),
    title=dict(font=dict(size=14)),
    template='plotly_white'
)

# Show the interactive plot
fig.show()
